In [129]:
!nvidia-smi

Thu May 16 09:45:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0              34W /  70W |    161MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# **Section 0: Imports and Install📢**




In [130]:
!pip install torchmetrics

In [131]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from torchmetrics.classification import BinaryAccuracy
import time

# **Section 1: Load Dataset📑**

## 1-1: Read Dataset

In [195]:
df = pd.read_csv('train.csv')
df.head(5)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
1,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
2,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
3,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
4,75.0,1,246,0,15,0,127000.00,1.2,137,1,0,10,1


### unneccessary cells

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       249 non-null    float64
 1   anaemia                   249 non-null    int64  
 2   creatinine_phosphokinase  249 non-null    int64  
 3   diabetes                  249 non-null    int64  
 4   ejection_fraction         249 non-null    int64  
 5   high_blood_pressure       249 non-null    int64  
 6   platelets                 249 non-null    float64
 7   serum_creatinine          249 non-null    float64
 8   serum_sodium              249 non-null    int64  
 9   sex                       249 non-null    int64  
 10  smoking                   249 non-null    int64  
 11  time                      249 non-null    int64  
 12  DEATH_EVENT               249 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 25.4 KB


In [134]:
df.shape

(249, 13)

##1-2: Train_Test_Split

In [135]:
# first, we should split out the y column from the x columns

X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']

In [138]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [139]:
X_train.shape, y_train.shape

((199, 12), (199,))

In [140]:
X_valid.shape, y_valid.shape

((50, 12), (50,))

##1-3: Preprocess

In [141]:
X_train = torch.FloatTensor(X_train.values) #the X values data type are already float64 ...
y_train = torch.FloatTensor(y_train.values) #the y column is already in the int64 datatype but thats just for more confidence

X_valid = torch.FloatTensor(X_valid.values)
y_valid = torch.FloatTensor(y_valid.values)

In [144]:
mu = X_train.mean(dim=0)
std = X_train.std(dim=0)

In [145]:
mu, std

(tensor([5.9621e+01, 3.8191e-01, 5.7404e+02, 3.8191e-01, 3.7774e+01, 3.6181e-01,
         2.5571e+05, 1.3656e+00, 1.3652e+02, 6.5327e-01, 3.2663e-01, 1.3182e+02]),
 tensor([1.1176e+01, 4.8708e-01, 9.5891e+02, 4.8708e-01, 1.1973e+01, 4.8174e-01,
         8.9907e+04, 1.0280e+00, 4.2851e+00, 4.7713e-01, 4.7016e-01, 7.6886e+01]))

In [146]:
X_train = (X_train - mu) / std
X_valid = (X_valid - mu) / std

### unneccessary cells

In [142]:
type(X_train), type(y_train)

(torch.Tensor, torch.Tensor)

In [143]:
X_train[:10]

tensor([[5.0000e+01, 0.0000e+00, 2.5000e+02, 0.0000e+00, 2.5000e+01, 0.0000e+00,
         2.6200e+05, 1.0000e+00, 1.3600e+02, 1.0000e+00, 1.0000e+00, 1.2000e+02],
        [6.0667e+01, 1.0000e+00, 1.5100e+02, 1.0000e+00, 4.0000e+01, 1.0000e+00,
         2.0100e+05, 1.0000e+00, 1.3600e+02, 0.0000e+00, 0.0000e+00, 1.7200e+02],
        [6.5000e+01, 0.0000e+00, 3.2600e+02, 0.0000e+00, 3.8000e+01, 0.0000e+00,
         2.9400e+05, 1.7000e+00, 1.3900e+02, 0.0000e+00, 0.0000e+00, 2.2000e+02],
        [5.0000e+01, 0.0000e+00, 4.8200e+02, 1.0000e+00, 3.0000e+01, 0.0000e+00,
         3.2900e+05, 9.0000e-01, 1.3200e+02, 0.0000e+00, 0.0000e+00, 1.0900e+02],
        [5.9000e+01, 0.0000e+00, 6.6000e+01, 1.0000e+00, 2.0000e+01, 0.0000e+00,
         7.0000e+04, 2.4000e+00, 1.3400e+02, 1.0000e+00, 0.0000e+00, 1.3500e+02],
        [7.0000e+01, 0.0000e+00, 6.9000e+01, 0.0000e+00, 4.0000e+01, 0.0000e+00,
         2.9300e+05, 1.7000e+00, 1.3600e+02, 0.0000e+00, 0.0000e+00, 7.5000e+01],
        [5.3000e+01, 0

In [147]:
X_train[:5]

tensor([[-0.8609, -0.7841, -0.3379, -0.7841, -1.0669, -0.7511,  0.0700, -0.3557,
         -0.1220,  0.7267,  1.4322, -0.1537],
        [ 0.0936,  1.2690, -0.4412,  1.2690,  0.1859,  1.3248, -0.6085, -0.3557,
         -0.1220, -1.3692, -0.6947,  0.5226],
        [ 0.4813, -0.7841, -0.2587, -0.7841,  0.0189, -0.7511,  0.4259,  0.3253,
          0.5781, -1.3692, -0.6947,  1.1469],
        [-0.8609, -0.7841, -0.0960,  1.2690, -0.6493, -0.7511,  0.8152, -0.4530,
         -1.0554, -1.3692, -0.6947, -0.2968],
        [-0.0556, -0.7841, -0.5298,  1.2690, -1.4845, -0.7511, -2.0656,  1.0062,
         -0.5887,  0.7267, -0.6947,  0.0414]])

##1-4: Dataloader

In [148]:
train_data = TensorDataset(X_train, y_train)
valid_data = TensorDataset(X_valid, y_valid)

In [152]:
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=8, shuffle=True)

### unneccessary cells

In [149]:
type(train_data), type(valid_data)

(torch.utils.data.dataset.TensorDataset,
 torch.utils.data.dataset.TensorDataset)

In [150]:
train_data.tensors

(tensor([[-0.8609, -0.7841, -0.3379,  ...,  0.7267,  1.4322, -0.1537],
         [ 0.0936,  1.2690, -0.4412,  ..., -1.3692, -0.6947,  0.5226],
         [ 0.4813, -0.7841, -0.2587,  ..., -1.3692, -0.6947,  1.1469],
         ...,
         [-0.4135, -0.7841, -0.5361,  ...,  0.7267,  1.4322, -0.5439],
         [ 0.9287, -0.7841,  0.6549,  ..., -1.3692, -0.6947,  0.8348],
         [ 0.0339, -0.7841, -0.4985,  ..., -1.3692, -0.6947, -0.4789]]),
 tensor([0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0.,
         0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,

In [153]:
train_loader.dataset

In [155]:
train_loader.batch_size

4

In [154]:
len(train_loader)

50

In [156]:
next(iter(train_loader))

[tensor([[-1.5767, -0.7841, -0.5319, -0.7841, -0.6493, -0.7511, -0.4528,  2.3681,
          -1.9889,  0.7267,  1.4322,  1.5371],
         [-0.4135,  1.2690, -0.4109, -0.7841,  0.6035, -0.7511,  0.0851, -0.1806,
           0.1114,  0.7267,  1.4322,  1.0298],
         [ 0.4813, -0.7841, -0.4245, -0.7841, -0.6493, -0.7511,  0.0366, -0.5502,
           0.3448, -1.3692, -0.6947,  0.7047],
         [ 0.7497,  1.2690, -0.4349,  1.2690,  1.8564, -0.7511, -0.5306, -0.3557,
           0.8115, -1.3692, -0.6947,  1.3680]]),
 tensor([0., 0., 0., 0.])]

In [157]:
next(iter(valid_loader))

[tensor([[-0.9504, -0.7841,  0.4150,  1.2690, -0.2317,  1.3248,  0.1367, -0.5502,
          -1.5222, -1.3692, -0.6947,  0.7177],
         [-0.5925,  1.2690,  0.0083, -0.7841,  0.6035, -0.7511,  0.5483, -0.2584,
           0.1114,  0.7267,  1.4322,  1.0038],
         [-0.5925, -0.7841, -0.5329,  1.2690,  1.8564, -0.7511,  1.2490, -0.5502,
          -0.3553,  0.7267, -0.6947, -1.4283],
         [-1.5767,  1.2690, -0.5089, -0.7841, -0.2317, -0.7511,  1.2156, -0.2584,
           0.5781,  0.7267,  1.4322,  0.8998],
         [-0.8609,  1.2690, -0.4234, -0.7841,  0.0189,  1.3248,  0.2257, -0.2584,
           0.1114,  0.7267, -0.6947, -1.5714],
         [ 0.1234,  1.2690, -0.5110, -0.7841,  0.1859,  1.3248, -0.2971, -0.4530,
           1.0449, -1.3692, -0.6947, -0.2838],
         [-1.2188,  1.2690, -0.2952, -0.7841, -0.2317, -0.7511,  1.0265, -0.4530,
           0.8115, -1.3692, -0.6947, -0.2968],
         [ 3.0762, -0.7841,  0.0083,  1.2690,  0.0189,  1.3248,  0.0851,  0.4517,
          -0.58

#**Section2: Define Model🧠**

##2-1: make your MLP Model

In [158]:
model = nn.Sequential(
    nn.Linear(12, 32),
    nn.ReLU(),
    nn.Linear(32, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
)

In [159]:
model

Sequential(
  (0): Linear(in_features=12, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=32, bias=True)
  (5): ReLU()
  (6): Linear(in_features=32, out_features=1, bias=True)
  (7): Sigmoid()
)

In [160]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1199,  0.0921,  0.0686, -0.2639,  0.2377,  0.0662, -0.1809, -0.1493,
          -0.0510, -0.0033,  0.1172,  0.1067],
         [ 0.1308,  0.1906,  0.1425,  0.2310,  0.1513, -0.2164, -0.1201,  0.2686,
          -0.0849,  0.1798, -0.2180,  0.0741],
         [-0.0143, -0.2442,  0.2442,  0.2332, -0.2669, -0.1770,  0.2822, -0.1185,
          -0.1966,  0.0440,  0.1428,  0.0186],
         [-0.0891, -0.0017, -0.2017, -0.2420, -0.1647, -0.1810, -0.1364, -0.2585,
          -0.1415, -0.2106, -0.2769,  0.2037],
         [-0.2531,  0.0425, -0.2128, -0.2002,  0.2489,  0.1389,  0.2696,  0.1051,
           0.0282, -0.0863,  0.0855, -0.0909],
         [-0.2458, -0.2601, -0.1566,  0.2301, -0.0417, -0.2226,  0.2231, -0.0549,
           0.1647,  0.0332, -0.0265, -0.0056],
         [-0.0254, -0.1459,  0.0906,  0.0370,  0.1499, -0.1286,  0.2191, -0.2015,
          -0.2777, -0.1665,  0.1717, -0.2075],
         [ 0.0407, -0.2378,  0.1099,  0.2886, -0.1876, -0.2236,  0.2124, -

In [161]:
X_batch, y_batch = next(iter(train_loader))

In [162]:
y_batch

tensor([0., 0., 0., 0.])

In [163]:
yp = model(X_batch)

In [164]:
yp

tensor([[0.5374],
        [0.5363],
        [0.5445],
        [0.5394]], grad_fn=<SigmoidBackward0>)

##2-2: Loss and Optimizer

In [165]:
loss_fn = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

##2-3: Bias for last layer(before train)

In [166]:
### last layer biases
last_layer_biases = model[-2].bias
print(last_layer_biases)

Parameter containing:
tensor([0.1473], requires_grad=True)


#**Section3: Device⚙️**

In [167]:
torch.cuda.is_available()

True

In [168]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [169]:
device

'cuda'

In [170]:
model = model.to(device)

#**Section4: Train Model🤓**

##4-1:Utils

In [171]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

##4-1: Torchmetrics

In [172]:
acc = BinaryAccuracy()

##4-2:Train Loop

In [174]:
num_epochs = 1000

loss_train_hist = []
loss_valid_hist = []

acc_train_hist = []
acc_valid_hist = []

for epoch in range(num_epochs):
  loss_train = AverageMeter()
  acc_train = BinaryAccuracy().to(device)
  for i, (inputs, targets) in enumerate(train_loader):
    inputs = inputs.to(device)
    targets = targets.to(device)

    outputs = model(inputs)

    loss = loss_fn(outputs.squeeze(), targets)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    loss_train.update(loss.item())
    acc_train(outputs.squeeze(), targets)

  with torch.no_grad():
    loss_valid = AverageMeter()
    acc_valid = BinaryAccuracy().to(device)
    for i, (inputs, targets) in enumerate(valid_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)
      loss = loss_fn(outputs.squeeze(), targets)

      loss_valid.update(loss.item())
      acc_valid(outputs.squeeze(), targets)

  loss_train_hist.append(loss_train.avg)
  loss_valid_hist.append(loss_valid.avg)

  acc_train_hist.append(acc_train.compute())
  acc_valid_hist.append(acc_valid.compute())

  if epoch % 10 == 0:
    print(f'Epoch {epoch}')
    print(f'Train: Loss = {loss_train.avg:.4}, Acc = {acc_train.compute():.4}')
    print(f'Valid: Loss = {loss_valid.avg:.4}, Acc = {acc_valid.compute():.4}')
    print()

Epoch 0
Train: Loss = 0.0008303, Acc = 1.0
Valid: Loss = 1.293, Acc = 0.8

Epoch 10
Train: Loss = 0.0007897, Acc = 1.0
Valid: Loss = 1.809, Acc = 0.8

Epoch 20
Train: Loss = 0.0007532, Acc = 1.0
Valid: Loss = 1.324, Acc = 0.8

Epoch 30
Train: Loss = 0.0007216, Acc = 1.0
Valid: Loss = 1.334, Acc = 0.8

Epoch 40
Train: Loss = 0.0006891, Acc = 1.0
Valid: Loss = 1.839, Acc = 0.8

Epoch 50
Train: Loss = 0.0006589, Acc = 1.0
Valid: Loss = 2.127, Acc = 0.8

Epoch 60
Train: Loss = 0.0006314, Acc = 1.0
Valid: Loss = 1.336, Acc = 0.8

Epoch 70
Train: Loss = 0.0006025, Acc = 1.0
Valid: Loss = 2.174, Acc = 0.8

Epoch 80
Train: Loss = 0.0005904, Acc = 1.0
Valid: Loss = 2.104, Acc = 0.8

Epoch 90
Train: Loss = 0.0005582, Acc = 1.0
Valid: Loss = 1.359, Acc = 0.8

Epoch 100
Train: Loss = 0.0005364, Acc = 1.0
Valid: Loss = 1.515, Acc = 0.8

Epoch 110
Train: Loss = 0.0005194, Acc = 1.0
Valid: Loss = 1.522, Acc = 0.8

Epoch 120
Train: Loss = 0.0005031, Acc = 1.0
Valid: Loss = 1.373, Acc = 0.8

Epoch 130


#**Section5:Result📈**

##5-1:Plot loss and acc

##5-2:Count param

In [178]:
total_params = 0
for param in model.parameters():
    total_params += param.numel()

print("Total parameters:", total_params)

Total parameters: 4641


##5-3: Bias for last layer(after train)

In [179]:
### last layer biases
last_layer_biases = model[-2].bias
print(last_layer_biases)

Parameter containing:
tensor([-0.0716], device='cuda:0', requires_grad=True)


#**Section6: Save and Load Model💰**

##6-1: Save

In [180]:
torch.save(model, 'model.pth')

##6-2: Load

In [182]:
mymodel = torch.load('model.pth')

#**Section7: Model Prediction🩺**

##7-1: Use test.csv

In [185]:
mymodel.eval()

Sequential(
  (0): Linear(in_features=12, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=32, bias=True)
  (5): ReLU()
  (6): Linear(in_features=32, out_features=1, bias=True)
  (7): Sigmoid()
)

In [187]:
test_data = pd.read_csv('test.csv')

In [188]:
inputs = torch.tensor(test_data.values).float().to(device)

In [189]:
with torch.no_grad():
  outputs = model(inputs)

In [190]:
predictions = (outputs > 0.5).int()

##7-2 Make predictions.csv file

In [191]:
predictions_array = predictions.cpu().numpy()

In [192]:
test_data['pred'] = predictions_array

In [193]:
test_data.to_csv('test_with_predictions.csv', index=False)